# R medians
https://www.kaggle.com/nigelcarpenter/grupo-bimbo-inventory-demand/r-medians/code

In [2]:
library('data.table')

In [3]:
raw <- "C:/Users/568377/Cognizant/Data Science/kaggle/Bimbo Bread/raw"

In [48]:
sub <- "C:/Users/568377/Cognizant/Data Science/kaggle/Bimbo Bread/submissions"

In [5]:
train <- fread(file.path(raw,"train.csv"), select = c('Cliente_ID','Producto_ID','Demanda_uni_equil'), colClasses=c(Cliente_ID="numeric",Producto_ID="numeric",Demanda_uni_equil="numeric"))

Read 74180464 rows and 3 (of 11) columns from 2.980 GB file in 00:01:39


In [6]:
head(train)

,Cliente_ID,Producto_ID,Demanda_uni_equil
1,15766,1212,3
2,15766,1216,4
3,15766,1238,4
4,15766,1240,4
5,15766,1242,3
6,15766,1250,5


In [7]:
sapply(train,class)

Cliente_ID       Producto_ID Demanda_uni_equil 
        "numeric"         "numeric"         "numeric"

set a table key to enable fast aggregations

In [8]:
setkey(train, Producto_ID, Cliente_ID)

In [24]:
median <- train[,median(Demanda_uni_equil)] # overall median

In [32]:
#calculate product overall median -> M2
median_Prod <- train[, median(Demanda_uni_equil), by = Producto_ID]
setnames(median_Prod, "V1","M2")

In [34]:
#calculate the client and product median -> M3
median_Client_Prod <- train[, median(Demanda_uni_equil),by = .(Producto_ID,Cliente_ID)]
setnames(median_Client_Prod,"V1","M3")

# Scoring

In [36]:
test <- fread(file.path(raw,"test.csv"), 
               select = c('id','Cliente_ID', 'Producto_ID'),
               colClasses=c(Cliente_ID="numeric",Producto_ID="numeric"))

Read 6999251 rows and 3 (of 7) columns from 0.234 GB file in 00:00:07


In [40]:
sapply(test, class)
setkey(test, Producto_ID, Cliente_ID)

id  Cliente_ID Producto_ID 
  "integer"   "numeric"   "numeric"

In [41]:
# add product and client and product medians
submit <- merge(test, median_Client_Prod, all.x = TRUE)
submit$M2 <- merge(test, median_Prod, by = "Producto_ID", all.x = TRUE)$M2

In [42]:
submit$Pred <- submit$M3

In [43]:
# where median by product and client is null use median by product (M2)
submit[is.na(M3)]$Pred <- submit[is.na(M3)]$M2

In [44]:
# where median by product is null use overall median
submit[is.na(Pred)]$Pred <- median

In [45]:
setnames(submit,"Pred","Demanda_uni_equil")

In [46]:
head(submit)

,Producto_ID,Cliente_ID,id,M3,M2,Demanda_uni_equil
1,41,681747,6236398,1430,70.0,1430.0
2,41,2284385,1678163,110,70.0,110.0
3,41,2482890,3458287,400,70.0,400.0
4,53,26372,2690233,138,119.5,138.0
5,53,1000006,1288534,NA,119.5,119.5
6,53,1216931,1127055,440,119.5,440.0


In [49]:
# Write out submission file.
# Any results you write to the current directory are saved as output.
write.csv(submit[,.(id,Demanda_uni_equil)],file.path(sub,"submission3.csv"), row.names = FALSE)

In [51]:
head(submit[,median(Demanda_uni_equil)])

[1] 3.5